In [12]:
import pandas as pd 
import numpy as np

import os

In [19]:
import re

def extract_final_answer(text):
    # Remove any artifacts like <<...>>
    text = re.sub(r'<<.*?>>', '', text)

    # Improved regex to catch full numbers like 1000, 105,000.00, etc.
    matches = re.findall(r'\b\d[\d,]*\.?\d*\b', text)

    if matches:
        cleaned = matches[-1].replace(',', '')
        try:
            return float(cleaned)
        except:
            return None
    
    return None

In [20]:
from datasets import load_dataset

In [23]:
dataset = load_dataset('gsm8k', 'main')

In [24]:
dataset = dataset['test']
dataset = dataset.shuffle(seed = 24).select(range(250))

In [35]:
correct_ans = pd.Series(dataset['answer']).apply(extract_final_answer)
correct_ans = [float(p) for p in correct_ans]

In [159]:
path = '../outputs/chatgpt-3.5-turbo/few_shot_cot_v1/results.csv'

In [160]:
results = pd.read_csv(path)

In [161]:
results['is_correct'].mean()

0.748

In [162]:
results['expected_answer'] = correct_ans
results['is_correct'] = (results['expected_answer'] == results['model_prediction']).astype(int)

In [163]:
results['is_correct'].mean()

0.82

In [164]:
results.to_csv(path, header = True, index = False)